## 얼굴 인식 캐릭터 씌우기

### Face Detection vs Face Recognition
* FaceDetection : 영상에서 사람 얼굴의 특징을 찾음 ex) 체온 체크   
* FaceRecognition : 그 얼굴이 누구의 얼굴인지를 알아냄 ex) 출결, 얼굴 분류  


In [51]:
import cv2
import mediapipe as mp

mp_face_detection = mp.solutions.face_detection # 얼굴 찾기 위한 모듈
mp_drawing = mp.solutions.drawing_utils # 찾은 얼굴의 특징을 그리기 위한 모듈

# 동영상 파일 불러오기
cap = cv2.VideoCapture('./images/face_video.mp4')

# 이미지 파일 불러오기
# dog
img_right = cv2.imread('./images/animal/dog_right_ear.png', cv2.IMREAD_UNCHANGED) # 105 x 105
img_left = cv2.imread('./images/animal/dog_left_ear.png', cv2.IMREAD_UNCHANGED) # 105 x 105
img_nose = cv2.imread('./images/animal/dog_nose.png', cv2.IMREAD_UNCHANGED) # 300 x 100

# leopard
# img_right = cv2.imread('./images/animal/leopard_right_ear.png', cv2.IMREAD_UNCHANGED) # 100 x 100
# img_left = cv2.imread('./images/animal/leopard_left_ear.png', cv2.IMREAD_UNCHANGED) # 100 x 100
# img_nose = cv2.imread('./images/animal/leopard_nose.png', cv2.IMREAD_UNCHANGED) # 300 x 110

# 투명도 포함한 4채널의 이미지를 3채널로 바꾸기
# 대상 이미지(3채널), x, y, 너비, 높이, 덮어씌울 이미지(4채널)
def overlay(image, x, y, w, h, overlay_img):
    alpha = overlay_img[:,:,3] / 255 # BGRA, alpha값을 0~1의 값으로 변환 (0: 투명, 1: 불투명)
    
    # BGR에 A값 추가
    for c in range(0, 3):
        image[y-h:y+h, x-w:x+w, c] = (overlay_img[:,:,c] * alpha) + (image[y-h:y+h, x-w:x+w, c] * (1-alpha))

'''
model_selection는 0 또는 1의 값을 가짐
0 : 2m 이내의 근거리
1 : 5m 이내의 거리

min_detection_confidence는 어느 정도의 적합도를 가진 것을 얼굴로 평가할지(0~1)
'''

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            break

        # 성능 향상
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image)

        # 이미지에 얼굴 특징 그리기
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.detections:
            for detection in results.detections: # 6개의 특징 : 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀
                # mp_drawing.draw_detection(image, detection)
                
                # 특정 위치 가져오기
                keypoints = detection.location_data.relative_keypoints
                right_eye, left_eye, nose = keypoints[0], keypoints[1], keypoints[2]
                
                h, w, _ = image.shape # 이미지의 높이, 너비 가져오기
                
                right_eye = (int(right_eye.x * w)-130, int(right_eye.y * h)-300) # 이미지 내의 실제 좌표
                left_eye = (int(left_eye.x * w)+130, int(left_eye.y * h)-300)
                nose = (int(nose.x * w), int(nose.y * h)+80)
                
                # dog
                overlay(image, *right_eye, 110, 120, cv2.resize(img_right, None, fx=2, fy=2))
                overlay(image, *left_eye, 110, 120, cv2.resize(img_left, None, fx=2, fy=2))
                overlay(image, *nose, 450, 150, cv2.resize(img_nose, None, fx=3, fy=3))
                
                # leopard
                # overlay(image, *right_eye, 100, 100, cv2.resize(img_right, None, fx=2, fy=2))
                # overlay(image, *left_eye, 100, 100, cv2.resize(img_left, None, fx=2, fy=2))
                # overlay(image, *nose, 450, 165, cv2.resize(img_nose, None, fx=3, fy=3))
                
                
        cv2.imshow('MediaPipe Face Detection', cv2.resize(image, None, fx=0.25, fy=0.25))
        
        if cv2.waitKey(1) == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()